<a href="https://colab.research.google.com/github/Alexandre-Delplanque/TFE-2020/blob/master/Model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OBJECT DETECTION - MODEL 1

## Importation des librairies et liaison à Mon Drive

In [0]:
#############################################################################
#                         OBJECT DETECTION MODEL 1                          #
#############################################################################

'''
Premier essai de modèle de détection d'objets, basé sur le tutoriel Pytorch :
https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html


'''

# --- Importation des librairies ---

# OS
import os
# PyTorch
import torch
import torchvision
# NumPy
import numpy as np
# Pillow
from PIL import Image
# Json
import json
# Matplotlib
import matplotlib.pyplot as plt
# OpenCV
import cv2
# Google drive
from google.colab import drive
drive.mount('/content/drive')


## Modification de "cocoeval.py"

In [0]:
# --- Modification du fichier 'cocoeval.py' afin de pouvoir utiliser la librairie ---

import re 

# Chemin d'accès vers le fichier cocoeval.py
fname = "/usr/local/lib/python3.6/dist-packages/pycocotools/cocoeval.py"

with open(fname) as f:
    # Lecture du fichier
    s = f.read()
    # Remplacement des lignes de code
    s = re.sub('self.iouThrs = (.+)',
               'self.iouThrs = np.linspace(.5, 0.95, (np.round((0.95 - .5) / .05) + 1).astype(np.int), endpoint=True)', s)
    s = re.sub('self.recThrs = (.+)',
               'self.recThrs = np.linspace(.0, 1.00, (np.round((1.00 - .0) / .01) + 1).astype(np.int), endpoint=True)', s)

# Ecriture dans le fichier
with open(fname, 'w') as f:
    f.write(s)

print(s)

## Importation des fonctions personnelles

In [0]:
!pip install bounding_box

In [0]:
!python "/content/drive/My Drive/Obj_Detection_M1/alex_utils.py"
!python "/content/drive/My Drive/Obj_Detection_M1/engine.py"
!python "/content/drive/My Drive/Obj_Detection_M1/utils.py"

In [0]:
# Perso
%cd '/content/drive/My Drive/Obj_Detection_M1/'
from alex_utils import *

## Importation des données

In [0]:
# --- Définition du dossier de travail ---

work_dir = "/content/drive/My Drive/Obj_Detection_M1/"

print(work_dir)

# Classes
animals_names = ["Bubale","Buffalo","Hippopotamus","Kob","Topi","Warthog","Waterbuck"]
animals_labels = [1, 2, 3, 4, 5, 6, 7]

# --- Sélection du processeur de calcul ---
use_cuda = torch.cuda.is_available()

if not use_cuda:
  print("WARNING: PYTORCH COULD NOT LOCATE ANY AVAILABLE CUDA DEVICE")
else:
  print("All good, a GPU is available.")

# --- Infos sur le GPU disponible ---
t = torch.cuda.get_device_properties(0).total_memory
t_GB = t/float((1024**3))
c = torch.cuda.memory_cached(0)
c_GB = c/float((1024**3))
a = torch.cuda.memory_allocated(0)
a_GB = a/float((1024**3))
f = c-a
f_GB = f/float((1024**3))

print("{:<30}{:<25}".format("Mémoire totale (GB) :",t_GB))
print("{:<30}{:<25}".format("Mémoire cachée (GB) :",c_GB))
print("{:<30}{:<25}".format("Mémoire allouée (GB) :",a_GB))
print("{:<30}{:<25}".format("Mémoire disponible (GB) :",f_GB))

In [0]:
# --- Téléchargment des données (GitHub) ---

%cd '/content/'

# Clone du repo contenant la data
git_repo_data = 'https://github.com/Alexandre-Delplanque/ObjectDetection_Model_2_MMdet.git' 
!git clone $git_repo_data

In [0]:
!ls /content/

from os.path import exists, join, basename, splitext

%cd /content
data_dir = os.path.abspath(splitext(basename(git_repo_data))[0])
print("Data path : {}".format(data_dir))

In [0]:
# --- Téléchargement des annotations (fichiers .json) ---

ann_path = data_dir + "/annotations"

with open(os.path.join(ann_path,'train_cocotype.json')) as json_file:
    train_js = json.load(json_file)

with open(os.path.join(ann_path,'val_cocotype.json')) as json_file:
    val_js = json.load(json_file)

with open(os.path.join(ann_path,'test_cocotype.json')) as json_file:
    test_js = json.load(json_file)


## Analyse des données

In [0]:
# --- Analyse du dataset ---

# Calcul du nombre d'animaux par set de données

# Train
dist_train = []
dist_class_train = []
for i in range(len(train_js['annotations'])):
    dist_class_train.append(train_js['annotations'][i]['category_id'])

# Validation
val_test = []
dist_class_val = []
for i in range(len(val_js['annotations'])):
    dist_class_val.append(val_js['annotations'][i]['category_id'])

# Test
dist_test = []
dist_class_test = []
for i in range(len(test_js['annotations'])):
    dist_class_test.append(test_js['annotations'][i]['category_id'])

# Print des infos du dataset
print('{:<15}{:<10}{:<10}'.format("DATASET","IMAGES","ANIMALS"))
print('-'*35)
print('{:<15}{:<10}{:<10}'.format("Train",len(train_js['images']),len(train_js['annotations'])))
print('{:<15}{:<10}{:<10}'.format("Validation",len(val_js['images']),len(val_js['annotations'])))
print('{:<15}{:<10}{:<10}'.format("Test",len(test_js['images']),len(test_js['annotations'])))
print('-'*35)

# Distribution des classes

import collections

# Calcul des distributions de fréquences
cl_train_counter = dict(collections.Counter(dist_class_train))
cl_test_counter = dict(collections.Counter(dist_class_test))

# Mise en graphique
plt.figure(1,figsize=(10,10))
bins = [x + 0.5 for x in range(0, len(cl_train_counter)+1)]
fig = plt.hist([dist_class_train, dist_class_val, dist_class_test], bins=bins, edgecolor='dimgray', color=['skyblue','khaki','ivory'])
axes = plt.gca()
axes.set_xlabel('Animal')
axes.xaxis.set_ticklabels([0]+animals_names)
axes.set_ylabel('Effectif')
plt.legend(['Entrainement','Validation','Test'], frameon=True)

## Pondération des classes

In [0]:
# --- Pondération des classes ---

# Vecteur de pondérations
'''
Pondération selon l'inverse de la fréquence de la classe dans le jeu
de données d'entrainement.

Résultats satisfaisants pour Kellenberger et al. (2018).

Attention le vecteur doit être un Tensor!
'''

n_train = []
for i in sorted(cl_train_counter.keys()):
    n_train.append(cl_train_counter[i])

n_train = torch.FloatTensor(n_train)
class_weights = torch.min(n_train)/n_train

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class_weights = class_weights.to(device) # Calcul via GPU

print(class_weights)

In [0]:
data_dir

## Installation d'albumentations (Data Augmentation)

In [0]:
# Installation de la librairie "albumentations" (data augmentation)
!pip install -U git+https://github.com/albu/albumentations

## Préparation du dataset

### Création : "MammalsDataset()"

In [0]:
# --- Création d'un dataset ---

from torch.utils.data import Dataset

class MammalsDataset(Dataset):

    def __init__(self, root, transforms=None, train=True, test=False):

        self.root = root
        self.transforms = transforms
        self.train = train
        self.test = test

        if train == True:

            with open(os.path.join(root,'annotations/train_cocotype.json')) as json_file:
                self.data = json.load(json_file)
            
            self.root_images = os.path.join(root,'train')

        elif train == False:

            with open(os.path.join(root,'annotations/val_cocotype.json')) as json_file:
                self.data = json.load(json_file)
            
            self.root_images = os.path.join(root,'val')
        
        if test == True:

            with open(os.path.join(root,'annotations/test_cocotype.json')) as json_file:
                self.data = json.load(json_file)

            self.root_images = os.path.join(root,'test')



    def __getitem__(self, idx):

        image_name = self.data['images'][idx]['file_name']
        image_id = self.data['images'][idx]['id']

        img_path = os.path.join(self.root_images,image_name)

        img = Image.open(img_path).convert("RGB")

        # Initialisation du dictionnaire
        target = {}

        boxes = []
        area = []
        labels = []

        for ann in range(len(self.data['annotations'])):
          if self.data['annotations'][ann]['image_id']== image_id:
            xmin = self.data['annotations'][ann]['bbox'][0]
            ymin = self.data['annotations'][ann]['bbox'][1]
            new_box = [xmin,
                       ymin,
                       xmin + self.data['annotations'][ann]['bbox'][2],
                       ymin + self.data['annotations'][ann]['bbox'][3]]
            boxes.append(new_box)
            labels.append(self.data['annotations'][ann]['category_id'])
            area.append(self.data['annotations'][ann]['area'])

        # Area
        area = torch.as_tensor(area, dtype=torch.float32)

        # Image ID
        image_id = torch.tensor([idx])

        # IsCrowd : on suppose = 0 pour tous
        nbr_obj = len(labels)
        iscrowd = torch.zeros(nbr_obj, dtype=torch.int64) 

        # Transformations
        if self.transforms is not None:

            # Image
            image_np = np.array(img)
            augmented  = self.transforms(image = image_np, bboxes = boxes)
            img = Image.fromarray(augmented['image'])

            # BBoxes
            boxes = augmented['bboxes']

        # Image en tensor
        img_to_tensor = torchvision.transforms.ToTensor()
        img = img_to_tensor(img)

        # Label en tensor
        labels = torch.as_tensor(labels, dtype=torch.int64)

        # Bounding Box en tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)

        # Remplissage du dictionnaire
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd  

        return img, target

    def __len__(self):
        return len(self.data['images'])

### Transformations (Data augmentation)

In [0]:
# --- Transformations ---

from torchvision import transforms

# Prétraitements
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.ToTensor()
    ]),
    'test': transforms.Compose([
        transforms.ToTensor()
    ]),
}

# Albumentations (data augmentation)
from albumentations import Compose, HorizontalFlip, VerticalFlip, BboxParams
from albumentations import Blur, RandomBrightness, RandomRotate90

# Initialisation de 'labels' pour éviter l'erreur de variable non-déclarée
# Remplacée par la list de labels dans le dataset lors du chrgt du dataloader
labels = [] 

data_transforms = {
    'train': Compose([        
        VerticalFlip(p=0.5),
        HorizontalFlip(p=0.5),
        RandomRotate90(p=0.5),
        RandomBrightness(limit=(-0.2,-0.2),p=0.5),
        RandomBrightness(limit=(0.2,0.2),p=0.5),
        Blur(blur_limit=(25, 25), p=0.5)
        ],

        # Obligatoire pour spécifier le format des BBox (x1,y1,x2,y2)
        # et le label_fields pour associer chaque bbox à un label
        bbox_params=BboxParams(format='pascal_voc', label_fields=labels)
    ),

    'val': Compose([
    ]),

    'test': Compose([
    ]),

}

## Dataloader

In [0]:
# --- Dataset et Dataloader ---

# Dataset
# -------------
train_dataset = MammalsDataset(root=data_dir, transforms=data_transforms['train'], train=True)
val_dataset = MammalsDataset(root=data_dir, transforms=data_transforms['val'], train=False)
test_dataset = MammalsDataset(root=data_dir, transforms=data_transforms['test'], train=False, test=True)

image_datasets = {'train': train_dataset,'val': val_dataset,'test': test_dataset}

# Dataloader
# -------------
# Besoin d'une fonction collate_fn() pour créer les batchs
# car les images ne contiennent pas le même nombre d'objets
def collate_fn(batch):
    return tuple(zip(*batch))

# Taille d'un batch
batch_size = 2

# Dataloader
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size=batch_size,
                                              shuffle=True,
                                              num_workers=0,
                                              collate_fn=collate_fn)
              for x in ['train', 'val', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}

# Classes
# -------------
class_names = ['Background'] + animals_names
num_class = len(class_names)


# Visualisation des classes et des tailles de datasets
print("Nombre de classes : ",num_class)
print("Noms des classes : ",class_names)
print("Espèces animales : ",animals_names)
print("Tailles des datasets : ",dataset_sizes)



## Visualisation d'un batch

### Fonction de visualisation : "BatchPlot()"

In [0]:
# --- Fonction de plot d'un batch, ou d'une prédiction ---

import matplotlib.patches as patches

def BatchPlot(images, targets, dataset_dic, labels_dic, save=False, predictions=None, img_names=False, legend=False):
    # Images et targets sur CPU
    images = list(image.to('cpu') for image in images)
    targets = [{k: v.to('cpu') for k, v in t.items()} for t in targets]

    # Transformation de l'image en PIL
    tensor_to_PIL = torchvision.transforms.ToPILImage(mode='RGB')

    # Couleurs des classes
    labels_color = ['b','r','c','m','orange','lime','aquamarine','peru','silver']

    # Noms des classes
    cls_names = []
    color_names = []

    if len(images)>1:
        # Création de fig et ax
        fig, ax = plt.subplots(1,len(images), figsize = (30, 20))

        # Plot des images et leurs bounding boxes
        for i in range(len(images)):

            cls_names = []
            color_names = []

            # Titre = nom de l'image
            if img_names == True:
                # ID de l'image
                ID = targets[i]['image_id']
                # Nom
                img_name = dataset_dic['images'][ID]['file_name']
                img_width = images[i].size()[2]
                img_height = images[i].size()[1]
                title = 'Image : {}\nObjet(s) : {}'.format(img_name,len(targets[i]['boxes']))
                ax[i].text(int(img_width/200), int(img_height/30), title, size='x-large', bbox=dict(boxstyle='round',facecolor='w', alpha=0.6))

            # Plot
            ax[i].axis('off')
            image = tensor_to_PIL(images[i])
            ax[i].imshow(image)

            for box in range(len(targets[i]['boxes'])):

                # Couleur bboxe et nom pour légende
                label = int(targets[i]['labels'][box])
                color = labels_color[label-1]
                color_names.append(color)
                name = labels_dic[label]
                cls_names.append(name)
            
                bboxe = targets[i]['boxes'][box]
                rect = patches.Rectangle((bboxe[0],bboxe[1]), bboxe[2]-bboxe[0], bboxe[3]-bboxe[1],linewidth=0.5,edgecolor=color,facecolor='none')
                ax[i].add_patch(rect)
            
            # Légende = boxes et couleur
            if legend == True:
                rects = []
                cls_names = [x for j, x in enumerate(cls_names) if cls_names.index(x) == j]
                color_names = [x for j, x in enumerate(color_names) if color_names.index(x) == j]
                for o in range(len(cls_names)):
                    rect = patches.Rectangle((0,0), 100, 100,linewidth=1,edgecolor=color_names[o],facecolor='none')
                    rects.append(rect)

                ax[i].legend(rects,cls_names)

            if predictions is not None:

                for box in range(len(predictions[i]['boxes'])):
                    bboxe = predictions[i]['boxes'][box]
                    rect = patches.Rectangle((bboxe[0],bboxe[1]), bboxe[2]-bboxe[0], bboxe[3]-bboxe[1],linewidth=0.5,edgecolor='r',facecolor='none')
                    ax[i].add_patch(rect)

    else:
        fig, ax = plt.subplots(1, figsize = (30, 20))
        ax.axis('off')
        image = tensor_to_PIL(images[0])
        ax.imshow(image)

        # Titre = nom de l'image
        if img_names == True:
            # ID de l'image
            ID = targets[0]['image_id']
            # Nom
            img_name = dataset_dic['images'][ID]['file_name']
            img_width = images[0].size()[2]
            img_height = images[0].size()[1]
            title = 'Image : {}\nObjet(s) : {}'.format(img_name,len(targets[0]['boxes']))
            ax.text(int(img_width/200), int(img_height/30), title, size='x-large', bbox=dict(boxstyle='round', facecolor='w', alpha=0.6))

        for box in range(len(targets[0]['boxes'])):

            # Couleur bboxe et nom pour légende
            label = int(targets[0]['labels'][box])
            color = labels_color[label-1]
            color_names.append(color)
            name = labels_dic[label]
            cls_names.append(name)
            
            bboxe = targets[0]['boxes'][box]
            rect = patches.Rectangle((bboxe[0],bboxe[1]), bboxe[2]-bboxe[0], bboxe[3]-bboxe[1],linewidth=1,linestyle='--',edgecolor=color,facecolor='none')
            ax.add_patch(rect)
        
        # Légende = boxes et couleur
        if legend == True:
            rects = []
            cls_names = [x for i, x in enumerate(cls_names) if cls_names.index(x) == i]
            color_names = [x for i, x in enumerate(color_names) if color_names.index(x) == i]
            for o in range(len(cls_names)):
                rect = patches.Rectangle((0,0), 100, 100,linewidth=1,edgecolor=color_names[o],facecolor='none')
                rects.append(rect)

            ax.legend(rects,cls_names)

        if predictions is not None:

            for box in range(len(predictions[0]['boxes'])):

                label = int(predictions[0]['labels'][box])
                color = labels_color[label-1]

                bboxe = predictions[0]['boxes'][box]
                rect = patches.Rectangle((bboxe[0],bboxe[1]), bboxe[2]-bboxe[0], bboxe[3]-bboxe[1],linewidth=1,edgecolor=color,facecolor='none')
                ax.add_patch(rect)

        if save == True:
            plt.savefig(os.path.join("/content/drive/My Drive/Obj_Detection_M1/Annotations/train",img_name)) 
        

In [0]:
# Visualisation d'un batch
%matplotlib inline
labels_dic = {
    1: 'Bubale',
    2: 'Buffle',
    3: 'Hippo',
    4: 'Cob',
    5: 'Topi',
    6: 'Phacochere',
    7: 'Cob a croissant'
}

images, targets = next(iter(dataloaders['train']))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]

BatchPlot(images, targets, train_js, labels_dic,save=False, img_names=True, legend=True)

In [0]:
%matplotlib inline
labels_dic = {
    1: 'Bubale',
    2: 'Buffle',
    3: 'Hippo',
    4: 'Cob',
    5: 'Topi',
    6: 'Phacochere',
    7: 'Cob a croissant'
}

# Dataloader spécifique à une image
img_name = "S_07_05_16_DSC00570.JPG"
dic = test_js

def SpecificImgDisplay(img_name, dataset, dic, predictions=False):
    for row in range(len(dic['images'])):
        if dic['images'][row]['file_name']==img_name:
            ID_img = int(row)

    specific_dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                  batch_size=1,
                                                  shuffle=False,
                                                  collate_fn=collate_fn,
                                                  sampler=torch.utils.data.SubsetRandomSampler([ID_img]),
                                                  num_workers=0)
                  for x in ['train', 'val', 'test']}

    # Obtention d'un batch
    images_b, targets_b = next(iter(specific_dataloaders[dataset]))
    images = list(image for image in images_b)
    targets = [{k: v for k, v in t.items()} for t in targets_b]

    # Plot
    if predictions is False:
        BatchPlot(images, targets, dic, labels_dic,save=False, img_names=True, legend=True)
    elif predictions is True:
        model.eval()
        images = list(image.to(device) for image in images_b)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets_b]
        out = model(images)

        # Non-Maximum Supression (NMS)
        nms_treshold = 0.35
        out_nms = [NMScustom(boxes=out[0]['boxes'], 
                            labels=out[0]['labels'], 
                            scores=out[0]['scores'], 
                            tresh=nms_treshold)]

        # Seuillage sur les scores
        cutoff_treshold = 0.20
        out_nms_cut = [CutOffScores(boxes=out_nms[0]['boxes'], 
                                    labels=out_nms[0]['labels'], 
                                    scores=out_nms[0]['scores'], 
                                    tresh=cutoff_treshold)]

        BatchPlot(images, targets, dic, predictions=out_nms_cut, labels_dic=labels_dic, save=False, img_names=True, legend=True)

In [0]:
img_name = input('Nom de l\'image (train) : ')
SpecificImgDisplay(img_name, dataset='train', dic=train_js, predictions=False)

In [0]:
img_name = "L_07_05_16_DSC00441.JPG"
SpecificImgDisplay(img_name, dataset='train', dic=train_js, predictions=False)

### Visualisation

In [0]:
# --- Visualisation d'un batch et ses bboxes ---
import time

%matplotlib inline
labels_dic = {
    1: 'Bubale',
    2: 'Buffle',
    3: 'Hippo',
    4: 'Cob',
    5: 'Topi',
    6: 'Phacochere',
    7: 'Cob a croissant'
}

# Obtention d'un batch  de données d'entrainement
for i, (images, targets) in enumerate(order_dataloaders['train']):

    # images = list(image for image in images)
    # targets = [{k: v for k, v in t.items()} for t in targets]

    # Plot
    print('Image [{}/{}]'.format(i+1,len(image_datasets['train'])))

    BatchPlot(images, targets, train_js, labels_dic, save=False, img_names=True, legend=True)

    # Passage à l'image suivante
    print("Appuyer sur ENTER pour passer à l'image suivante...")
    input()

    plt.close()

## Chargement du modèle et transfer learning

In [0]:
# --- Chargement du modèle ---

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Chargement d'un modèle pré-entrainé (COCO)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# --- Ajustement du classifier ---

# Nombre d'inputs du classifier
num_features = model.roi_heads.box_predictor.cls_score.in_features

# Remplacement du classifier
model.roi_heads.box_predictor = FastRCNNPredictor(num_features, num_class)

In [0]:
# --- Entrainement du modèle ---
import torch.nn as nn
import torch.optim as optim

# Instanciation de la fonction de coût sous forme d'un objet
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(model.parameters(), lr=0.01, momentum=0.9,weight_decay=0.0005)

optimizer_ft = optim.SGD([
                          {'params':model.backbone.parameters(), 'lr':0.001},
                          {'params':model.roi_heads.box_predictor.parameters(), 'lr':0.005}
                         ], lr=0.005, momentum=0.9,weight_decay=0.0005)



# Diminution du LR de l toutes les k epochs
l = 0.1
k = 7
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=k, gamma=l)

# Modèle poussé sur le GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Entrainement

In [0]:
import time

%cd '/content/drive/My Drive/Obj_Detection_M1/'
import utils
from engine import train_one_epoch, evaluate

num_epochs = 30

for epoch in range(num_epochs):
    # Entrainement sur une epoch et print tous les 50 itérations
    train_one_epoch(model, optimizer_ft, dataloaders['train'], device, epoch, print_freq=50)
    # Mis à jour du learning rate
    exp_lr_scheduler.step()
    # Evaluation sur le dataset de test
    evaluate(model, dataloaders['val'], device=device)
    # Vider le cache du GPU
    torch.cuda.empty_cache()

checkpoint_file = work_dir + "Checkpoints/NEW_M1_OD_30_DA_2_ALL.pth"
torch.save(model.state_dict(), checkpoint_file)

## Chargement des paramètres d'un modèle entrainé

In [0]:
# --- Chargement des paramètres du modèle ---

checkpoint_file = work_dir + "Checkpoints/NEW_M1_OD_30_DA_2_ALL.pth"

# Sauver
#torch.save(model.state_dict(), checkpoint_file)

# Loader
model.load_state_dict(torch.load(checkpoint_file))

# Téléchargement
# from google.colab import files
# files.download(checkpoint_file)


## Prédictions d'un batch aléatoire

### Prédictions

In [0]:
# --- Prédictions d'un batch aléatoire ---
torch.cuda.empty_cache()

# Dataloader
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size=1,
                                              shuffle=True,
                                              num_workers=0,
                                              collate_fn=collate_fn)
              for x in ['train', 'val', 'test']}

model.eval()

imgs_test, targs_test = next(iter(dataloaders['test']))
imgs_test = list(image.to(device) for image in imgs_test)
targs_test = [{k: v.to(device) for k, v in t.items()} for t in targs_test]

out = model(imgs_test)
ground_truth = targs_test

# ID de l'image
ID = int(targs_test[0]['image_id'])
print("ID : {}".format(ID))
# Path
path = os.path.join(data_dir,"test/" + test_js['images'][ID]['file_name'])
print("Path : {}".format(path))

### Post-processing

In [0]:
# --- Post-processing ---

# Non-Maximum Supression (NMS)
nms_treshold = 0.35
out_nms = [NMScustom(boxes=out[0]['boxes'], 
                     labels=out[0]['labels'], 
                     scores=out[0]['scores'], 
                     tresh=nms_treshold)]

# Seuillage sur les scores
cutoff_treshold = 0.20
out_nms_cut = [CutOffScores(boxes=out_nms[0]['boxes'], 
                            labels=out_nms[0]['labels'], 
                            scores=out_nms[0]['scores'], 
                            tresh=cutoff_treshold)]

# Print des résultats
print('Résultats du post-processing')
print('-'*50)
print('{:<17}{:<7}{:<7}{:<7}{:<7}'.format(' ','GT','Preds','NMS','Cut-off'))
print('-'*50)
print('{:<17}{:<7}{:<7}{:<7}{:<7}'.format('Nbre de bbox :',len(ground_truth[0]['boxes']),len(out[0]['boxes']),len(out_nms[0]['boxes']),len(out_nms_cut[0]['boxes'])))
print('-'*50)


### Visualisation

In [0]:
# --- Visualisation des prédictions ---

%matplotlib inline
#BatchPlot(imgs_test, targs_test, out_nms_cut)
BatchPlot(imgs_test, targs_test, test_js, predictions=out_nms_cut, labels_dic=labels_dic ,save=False, img_names=True, legend=True)


## Matrice de confusion du jeu de test

In [0]:
# --- Matrice de confusion globale ---
''' Matrice de confusion de l'ensemble du jeu de test '''
from sklearn.metrics import confusion_matrix

# Dataloader ordonné 
order_dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                    batch_size=1,
                                                    shuffle=False,
                                                    collate_fn=collate_fn,
                                                    sampler=torch.utils.data.SequentialSampler(image_datasets[x]),
                                                    num_workers=0)
                      for x in ['train', 'val', 'test']}

torch.cuda.empty_cache()


model.eval()
print_every = 1

for i, (imgs_test, targs_test) in enumerate(order_dataloaders['test']):

    # GPU
    imgs_test = list(image.to(device) for image in imgs_test)
    targs_test = [{k: v.to(device) for k, v in t.items()} for t in targs_test]

    # Prédictions et ground-truth
    out = model(imgs_test)
    ground_truth = targs_test

    # S'il n'y a aucune bbox prédite => FN
    if len(out[0]['boxes'])==0:
        for k in range(len(ground_truth[0]['labels'])):
            res = np.array([int(ground_truth[0]['labels'][k]), int(0)])
            res = res.reshape(1,2)
            matches = np.concatenate((matches, res))
        continue

    # Non-Maximum Supression (NMS)
    nms_treshold = 0.35
    out_nms = [NMScustom(boxes=out[0]['boxes'], 
                        labels=out[0]['labels'], 
                        scores=out[0]['scores'], 
                        tresh=nms_treshold)]

    # Seuillage sur les scores
    cutoff_treshold = 0.20
    out_nms_cut = [CutOffScores(boxes=out_nms[0]['boxes'], 
                                labels=out_nms[0]['labels'], 
                                scores=out_nms[0]['scores'], 
                                tresh=cutoff_treshold)]

    # Matching des prédictions avec la ground-truth
    tresh = 0.25
    res = PredGTMatching(ground_truth, out_nms_cut, tresh)

    # Variable contenant les matching pred-GT
    if i == 0:
        matches = res
    else:
        matches = np.concatenate((matches, res))

    # ID de l'image
    ID = int(targs_test[0]['image_id'])

    if i % print_every == 0 or i == len(dataloaders['test'])-1:
        print('='*50)
        print('Image [{:<3}/{:<3}] traitée'.format(i+1, len(test_js['images'])))
        print('Nom de l\'image : {}'.format(test_js['images'][ID]['file_name']))
        print(' ')
        # Print des résultats
        print('Résultats du post-processing')
        print('-'*50)
        print(' ')
        print('{:<17}{:<7}{:<7}{:<7}{:<7}'.format(' ','GT','Preds','NMS','Cut-off'))
        print('-'*50)
        print('{:<17}{:<7}{:<7}{:<7}{:<7}'.format('Nbre de bbox :',len(ground_truth[0]['boxes']),len(out[0]['boxes']),len(out_nms[0]['boxes']),len(out_nms_cut[0]['boxes'])))
        print('-'*50)
        print(' ')
        # Matrice de confusion
        conf_matrix = confusion_matrix(res[:,0],res[:,1],labels=[0,1,2,3,4,5,6,7])
        print('Matrice de confusion')
        print('-'*50)
        print(' ')
        print(conf_matrix)

    torch.cuda.empty_cache()

In [0]:
# Construction de la matrice de confusion
from sklearn.metrics import confusion_matrix, classification_report

truth = matches[:,0]
predicted = matches[:,1]

conf_matrix = confusion_matrix(truth,predicted,labels=[0,1,2,3,4,5,6,7])

print('Matrice de confusion globale (test)')
print('-'*65)
print(' ')
print(conf_matrix)
print(' ')
print('Métriques')
print('-'*65)
print(' ')
print(classification_report(truth, predicted, target_names=class_names, digits=8))

In [0]:
# --- Analyse d'erreur ---
img_name = input('Nom de l\'image : ')
SpecificImgDisplay(img_name=img_name, dataset='test', dic=test_js, predictions=True)